# Custom Choropleth Maps with Plotly/Mapbox (full version)

## 1. Data

REST API requests, dataframes, GeoJSONs and point JSONs

In [2]:
import requests
import pandas as pd
import json

## Requests
supply_r = requests.get('https://peridash.ml/api/supply-areas/')
measurements_r = requests.get('https://peridash.ml/api/measurements/')
epsas_r = requests.get('https://peridash.ml/api/epsas/')

## Dataframes
measurements_df = pd.read_json(measurements_r.text)
epsas_df = pd.read_json(epsas_r.text)
df = pd.merge(measurements_df, epsas_df, left_on='epsa', right_on='url')

## GeoJSONs
supply_json = json.loads(supply_r.text)

epsa_jsons = []
for feature in supply_json['features']:
    epsa_json = dict(
        type='FeatureCollection',
        features=[feature]
    )
    epsa_jsons.append(epsa_json)
    
## Points JSONs
with open('points.json', encoding='utf8') as f:
    points_json = json.load(f)

## 2. Figure Specification

In [35]:
'CODE-16'.split('-')[0]

'CODE'

In [41]:
val_year = 2016

def get_display_value(epsa_code):
    if epsa_code in list(df.code):
        val = df[(df.code==epsa_code)&(df.year==val_year)].ind8.iloc[0]
        if np.isnan(val):
            val = 0.0
    else:
        val = 0.0
    return val

[get_display_value(code.split('-')[0]) for code in list(points_json.keys())]

[81.71,
 0.0,
 81.15,
 97.53,
 0.0,
 89.88,
 96.76,
 98.89,
 94.8,
 90.1,
 92.22,
 0.0,
 93.98,
 0.0,
 98.05,
 66.67,
 0.0,
 48.05,
 97.53,
 88.27,
 99.75,
 89.5,
 0.0,
 97.96,
 66.97,
 91.69,
 95.43,
 83.57,
 71.72,
 96.91,
 91.04,
 79.35,
 71.72,
 92.36,
 0.0,
 0.0,
 86.49,
 98.53,
 97.02,
 97.02,
 97.02,
 96.91,
 99.66,
 96.91,
 84.35,
 93.61,
 0.0,
 0.0,
 0.0,
 0.0,
 100.0,
 100.0,
 100.0,
 100.0,
 0.0,
 87.61,
 77.85,
 50.93,
 90.84,
 0.0,
 89.95,
 94.78,
 96.91,
 96.91,
 96.91,
 96.91,
 96.91,
 96.91,
 96.91,
 96.91,
 96.91,
 96.91,
 96.91,
 96.91,
 96.91,
 96.91,
 96.91,
 96.91,
 96.91,
 96.91,
 96.91,
 96.91,
 99.43,
 91.65,
 91.2,
 92.95,
 77.8,
 91.29,
 97.52]

In [47]:
import numpy as np

min_value = min(list(df[df.year == 2016].ind8))

min_ranges = [0.0, 0.35, 0.5, 0.6, 0.7]
max_ranges = [0.35, 0.5, 0.6, 0.7, 1.1]
min_colors = [[220,220,220],[106,137,247],[90,120,245],[70,100,245],[40,60,190]]
max_colors = [[106,137,247],[90,120,245],[70,100,245],[40,60,190],[5,10,172]]

val_label = 'V8'
val_unit = '%'
val_year = 2016

def interpolate(c1, c2, f):
    r = c1[0] + (c2[0]-c1[0]) * f
    g = c1[1] + (c2[1]-c1[1]) * f
    b = c1[2] + (c2[2]-c1[2]) * f
    return [r,g,b]

def get_layer_code(layer_index):
    code = epsa_jsons[layer_index]['features'][0]['properties']['code']
    code = code.split('-')[0]
    return code

def get_layer_color(epsa_code, min_value):
    if epsa_code in list(df.code):
        percentage = df[(df.code==epsa_code)&(df.year==val_year)].ind8.iloc[0]
        if np.isnan(percentage):
            percentage = 0 
    else:
        percentage = 0
        
    factor = (percentage - min_value)/(100 - min_value) # Biased factor for more contrast
    
    color1 = min_colors[0]
    color2 = min_colors[1]
    real_factor = 0.0
    for range_min, range_max, c_min, c_max in zip(min_ranges, max_ranges, min_colors, max_colors):
        if range_min <= factor < range_max:
            color1 = c_min
            color2 = c_max
            real_factor = (factor - range_min) / (range_max - range_min)
    
    c = interpolate(color1, color2, real_factor)
    color_string = f'rgb({str(c[0])[:6]},{str(c[1])[:6]},{str(c[2])[:6]})'
    return(color_string)


def get_display_value(epsa_code):
    if epsa_code in list(df.code):
        val = df[(df.code==epsa_code)&(df.year==val_year)].ind8.iloc[0]
        if np.isnan(val):
            val = 0.0
    else:
        val = 0.0
    return val

layers = [
    dict(
        sourcetype = 'geojson',
        source = epsa_jsons[k],
#         below = "water",
        type = 'fill',   
        color = get_layer_color(get_layer_code(k), min_value),
        opacity = 0.8
    ) for k in range(len(epsa_jsons))
]

p_lats = []
p_lons = []
p_texts = []

for code in list(points_json.keys()):
    epsa_p = points_json[code]
    n = len(epsa_p['lats'])
    p_lats += epsa_p['lats']
    p_lons += epsa_p['lons']
    
    clean_code = code.split('-')[0]
    value = get_display_value(clean_code)
    display_text = f'{clean_code}<br>{val_label}: {value} {val_unit}'
    p_texts += [display_text for k in range(n)]
    
scatter_dict = dict(
    type = 'scattermapbox',
    mode = 'markers',
    
    lat = p_lats, 
    lon = p_lons,
    text = p_texts,
    
    marker = dict(
        size = 50,
        opacity = 0,
        color= 'rgb(37,52,148)'
    ),
    showlegend = False,
    hoverinfo = 'text',
)

mapbox_access_token = 'pk.eyJ1Ijoic2VyZ2lvLWNodW1hY2Vyby1maSIsImEiOiJjamswOTUzeHkwMDk0M3dvNnJoeTByZGlpIn0.3mmjpLwDrIUcdJTowlCd1A'

layout = dict(
    title = 'Áreas de Cobertura de las EPSA Reguladas',
    font = dict(family='Balto'),
    autosize = False,
    width = 1200,
    height = 700,
    hovermode = 'closest',

    mapbox = dict(
        accesstoken = mapbox_access_token,
        layers = layers,
        bearing = 0,
        center = dict( 
            lat = -17.610907366555434, 
            lon = -63.13396632812757,
        ),
        pitch = 0,
        zoom = 8,
        style = 'light'
    ) 
)

fig = dict(data=[scatter_dict], layout=layout)

## 3. Gimme the Map!

In [48]:
import plotly.plotly as py
py.iplot(fig, filename='mapa-coberturas')